# FFM practice

Ref:
- https://github.com/princewen/tensorflow_practice/blob/master/recommendation/recommendation-FFM-Demo/FFM_model.py
- https://www.jianshu.com/p/781cde3d5f3d

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

In [3]:
input_x_size = 20  # 每个样本输入的维度（20个特征，10个属于field1，10个属于field2）
field_size = 2

vector_dimension = 3

total_plan_train_steps = 1000
# 使用SGD，每一个样本进行依次梯度下降，更新参数
batch_size = 1

all_data_size = 1000  # 训练样本个数

lr = 0.01

MODEL_SAVE_PATH = "TFModel"
MODEL_NAME = "FFM"

In [4]:
# 随机生成一些模拟数据
def gen_data():
    labels = [-1,1]
    y = [np.random.choice(labels,1)[0] for _ in range(all_data_size)]
    x_field = [i // 10 for i in range(input_x_size)]  # 前10个值为0，后10个值为1
    x = np.random.randint(0,2,size=(all_data_size,input_x_size))
    return x,y,x_field

In [5]:
# 定义权重项（bias项的权重，一维特征的权重，交叉特征的权重

# 交叉特征的权重（ 每个v的长度为vector_dimension维，共有input_x_size * field_size 个v向量
def createTwoDimensionWeight(input_x_size,field_size,vector_dimension):
    weights = tf.truncated_normal([input_x_size,field_size,vector_dimension])

    tf_weights = tf.Variable(weights)

    return tf_weights

def createOneDimensionWeight(input_x_size):
    weights = tf.truncated_normal([input_x_size])
    tf_weights = tf.Variable(weights)
    return tf_weights

def createZeroDimensionWeight():
    weights = tf.truncated_normal([1])
    tf_weights = tf.Variable(weights)
    return tf_weights


![ffm](https://codingcat.cn/uploads/images/4f60aed0-22d7-11e9-9b7d-fa163e7a698c.png)

In [6]:
# 根据ffm的公式，来计算估计值，对照公式看很容易理解

# 重点学习一下这个方法里面用到的tf的内置方法及作用
def inference(input_x,input_x_field,zeroWeights,oneDimWeights,thirdWeight):
    """计算回归模型输出的值"""

    secondValue = tf.reduce_sum(tf.multiply(oneDimWeights,input_x,name='secondValue'))

    firstTwoValue = tf.add(zeroWeights, secondValue, name="firstTwoValue")

    thirdValue = tf.Variable(0.0,dtype=tf.float32)
    input_shape = input_x_size

    for i in range(input_shape):
        featureIndex1 = i
        fieldIndex1 = int(input_x_field[i]) # 该feature所属field
        for j in range(i+1,input_shape):
            featureIndex2 = j
            fieldIndex2 = int(input_x_field[j]) # 该feature所属field
            # convert_to_tensor 作用：将python数据转换为tf的tensor
            vectorLeft = tf.convert_to_tensor([[featureIndex1,fieldIndex2,i] for i in range(vector_dimension)])
            # gather_nd 作用：根据索引，从向量中把索引位置的值提取出来，组成新的向量
            weightLeft = tf.gather_nd(thirdWeight,vectorLeft)
            # squeeze 作用：删除所有长度为1的维度，例如某个三维向量各维度长度分别为[1,2,3] squeeze后得到的向量，维度为[2,3]
            # 此处的squeeze相当于把一个[1,1,3]的三维挤成一个1维向量 [3]
            weightLeftAfterCut = tf.squeeze(weightLeft)

            vectorRight = tf.convert_to_tensor([[featureIndex2,fieldIndex1,i] for i in range(vector_dimension)])
            weightRight = tf.gather_nd(thirdWeight,vectorRight)
            weightRightAfterCut = tf.squeeze(weightRight)
            
            # reduce_sum 压缩求和，例如 reduct_sum([[1,1],[2,3]]) = 1+1+2+3 = 7
            tempValue = tf.reduce_sum(tf.multiply(weightLeftAfterCut,weightRightAfterCut))

            indices2 = [i]
            indices3 = [j]

            xi = tf.squeeze(tf.gather_nd(input_x, indices2))
            xj = tf.squeeze(tf.gather_nd(input_x, indices3))

            product = tf.reduce_sum(tf.multiply(xi, xj))

            secondItemVal = tf.multiply(tempValue, product)

            tf.assign(thirdValue, tf.add(thirdValue, secondItemVal))

    return tf.add(firstTwoValue,thirdValue)


In [7]:
# 构建模型

global_step = tf.Variable(0,trainable=False)
# 生成模拟数据
trainx,trainy,trainx_field = gen_data()

# 定义输出placeholder
input_x = tf.placeholder(tf.float32,[input_x_size ])
input_y = tf.placeholder(tf.float32)


lambda_w = tf.constant(0.001, name='lambda_w')
lambda_v = tf.constant(0.001, name='lambda_v')

# 初始化权重参数
zeroWeights = createZeroDimensionWeight()

oneDimWeights = createOneDimensionWeight(input_x_size)

thirdWeight = createTwoDimensionWeight(input_x_size,  # 创建二次项的权重变量
                                       field_size,
                                       vector_dimension)  # n * f * k

# 计算得到 y_
y_ = inference(input_x, trainx_field,zeroWeights,oneDimWeights,thirdWeight)


In [8]:
# 定义损失函数及正则化项
l2_norm = tf.reduce_sum(
    tf.add(
        tf.multiply(lambda_w, tf.pow(oneDimWeights, 2)),
        tf.reduce_sum(tf.multiply(lambda_v, tf.pow(thirdWeight, 2)),axis=[1,2])
    )
)

loss = tf.log(1 + tf.exp(input_y * y_)) + l2_norm

In [9]:
# 定义train step
train_step = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(loss)

In [10]:
# train
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(total_plan_train_steps):
        for t in range(all_data_size):
            input_x_batch = trainx[t]
            input_y_batch = trainy[t]
            predict_loss,_, steps = sess.run([loss,train_step, global_step],
                                           feed_dict={input_x: input_x_batch, input_y: input_y_batch})

            print("After  {step} training   step(s)   ,   loss    on    training    batch   is  {predict_loss} "
                  .format(step=steps, predict_loss=predict_loss))

            saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=steps)
            writer = tf.summary.FileWriter(os.path.join(MODEL_SAVE_PATH, MODEL_NAME), tf.get_default_graph())
            writer.close()


After  0 training   step(s)   ,   loss    on    training    batch   is  [0.71496004] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [1.0718213] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [0.47387955] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [1.738892] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [0.58538455] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [0.8071034] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [3.3097954] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [1.0471672] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [0.90875643] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [2.7981732] 
After  0 training   step(s)   ,   loss    on    training    batch   is  [3.7000327] 
After  0 training   step(s)   ,   loss    on    training    ba

KeyboardInterrupt: 